In [1]:
!nvidia-smi

Sun Feb  6 05:46:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --ignore-installed PyYAML
import yaml
print(yaml.__version__)

     |████████████████████████████████| 596 kB 5.5 MB/s 
6.0


In [3]:
import os
import shutil
import json

from tqdm import tqdm
from glob import glob

from PIL import Image
import torch
import random
import numpy as np

In [4]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!git clone https://github.com/ultralytics/yolov5 
!pip -q install -r /content/yolov5/requirements.txt
!pip -q install wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 10878, done.
remote: Total 10878 (delta 0), reused 0 (delta 0), pack-reused 10878
Receiving objects: 100% (10878/10878), 10.89 MiB | 30.39 MiB/s, done.
Resolving deltas: 100% (7525/7525), done.
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 180 kB 65.9 MB/s 
     |████████████████████████████████| 143 kB 54.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


## Dataset 불러오기

In [8]:
# Test Image
if not os.path.exists('/content/tmp_images'):
    os.mkdir('/content/tmp_images')

!unzip -q /content/drive/MyDrive/AI_Factory/data_test/[원천]돼지_bbox.zip -d /content/tmp_images/
!unzip -q /content/drive/MyDrive/AI_Factory/data_test/[원천]소_bbox.zip -d /content/tmp_images/

if not os.path.exists('/content/images'):
    os.mkdir('/content/images')
if not os.path.exists('/content/images/test'):
    os.mkdir('/content/images/test')

source = '/content/tmp_images/'
destination = '/content/images/test'
source_file = os.listdir(source)

for f in source_file:
    shutil.move(source + f, destination)

In [9]:
test_image = glob('/content/images/test/*.jpg')
print('Test Data의 개수', len(test_image))

Test Data의 개수 2058


In [10]:
# 리스트 이름을 txt파일로 생성
test_list = glob('/content/images/test/*.jpg')
 
with open('/content/test.txt', 'w') as f:
  f.write('\n'.join(test_list) + '\n')

In [11]:
 # coco.yaml을 바탕으로 우리 데이터셋 이름대로 변경하여 data.yaml로 저장
with open('/content/yolov5/data/coco.yaml', 'r') as f:
	data = yaml.load(f, Loader=yaml.FullLoader)
     
data['path'] = '/content' 
data['train'] = 'train.txt'
data['val'] = 'valid.txt'
data['test'] = 'test.txt'
data['nc'] = 2
data['names'] = ['pig','cow'] # pig랑 cow랑 반대로 나와서그럼 -> ['pig', 'cow']로 바꿔주기
 
with open('/content/data.yaml', 'w') as f:
	yaml.dump(data, f)

## Inference

In [ ]:
!python /content/yolov5/detect.py --source 'images/test' --weights '/content/drive/MyDrive/AI_Factory/trained_model/m6_pretrained_full_b10_e20_hyp_tuning_v1.pt' --data './data.yaml' --img 1280 --save-conf --save-txt --iou-thres 0.68 --conf-thres 0.001

## 제출 Dataset 만들기

In [ ]:
import os
from glob import glob
import pandas as pd

labels_path = 'yolov5/runs/detect/exp/labels/*'
labels_data = glob(labels_path)
images_path = 'images/test/*'
images_data = []
for image in glob(images_path):
    image_name = image.split('/')[-1].split('.')[0]
    for label in labels_data:
        if image_name in label:
            images_data.append(image)
            break

labels_data.sort()
images_data.sort()

result_list = []
for image, label in tqdm(zip(images_data, labels_data)):
    file_name = label.split('/')[-1].split('.')[0]
    if file_name == 'livestock_cow_bbox_014379':
        continue
    
    with open(label, 'r') as f:
        data = f.readlines()
    
    max_width, max_height = Image.open(image).size
    for target in data:
        id, x, y, w, h, conf = map(float, target.strip().split())
        id = 'cow' if id == 1 else 'pig'
        x *= max_width
        y *= max_height
        w *= max_width
        h *= max_height

        x_min = x - w/2 if x - w/2 > 0 else 0
        y_min = y - h/2 if y - h/2 > 0 else 0
        x_max = x + w/2 if x + w/2 < max_width else max_width
        y_max = y + h/2 if y + h/2 < max_height else max_height
        result_list.append([file_name + '.jpg', id, conf, x_min, x_max, y_min, y_max])

df_result = pd.DataFrame(result_list, columns=['ImageID','LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
df_result.to_csv('result_m6_pretrained_full_b10_e20_hyp_tuning_v1_linear_최종.csv', index=False, encoding='utf-8')

In [ ]:
source = '/content/result_m6_pretrained_full_b10_e20_hyp_tuning_v1_linear_최종.csv'
destination = '/content/drive/MyDrive/AI_Factory/result_m6_pretrained_full_b10_e20_hyp_tuning_v1_linear_최종.csv'
shutil.move(source,destination)

## Score Check

In [ ]:
source = '/content/drive/MyDrive/AI_Factory/answer.csv'
destination = '/content/'
shutil.copy(source, destination)

In [ ]:
!pip -q install map-boxes

In [ ]:
import os
from glob import glob
from sys import argv
import pandas as pd
from map_boxes import mean_average_precision_for_boxes


ans = pd.read_csv('answer.csv')
pred = pd.read_csv('result_m6_pretrained_full_b10_e10_hyp_tuning.csv')

pred = pred[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']]

ans = ans.sort_values(by=['ImageID'], ignore_index=True)
pred = pred.sort_values(by=['ImageID'], ignore_index=True)

ans_images = list(ans['ImageID'].unique())
for image_name in ans_images:
    # width, height = ans[ans['ImageID'] == image_name][['Width', 'Height']].values[0]
    width, height = [1920, 1080] if 'cow' in image_name else [1280, 960]
    pred.loc[pred['ImageID'] == image_name, 'XMin':'XMax'] /= width
    pred.loc[pred['ImageID'] == image_name, 'YMin':'YMax'] /= height

ans_arr = ans[['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']].values
pred_arr = pred[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']].values


mean_ap, ap = mean_average_precision_for_boxes(ans_arr, pred_arr, iou_threshold=0.75, verbose=False)

mean_ap = round(mean_ap, 5)

print(f'score:{mean_ap}')